In [1]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC
from pyspark.ml.functions import vector_to_array
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from math import *
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import LDA


In [2]:
spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/13 11:36:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/13 11:37:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/13 11:37:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [9]:
def rms_rs_files(path_detected) :
    
    print(path_detected)
    
    df = spark.read.option("header", "true").option("delimiter", ";").csv(path_detected)
    
    cl0,cl1,cl2 = df.columns[0],df.columns[1],df.columns[2]
    
    df_final = df.groupBy(cl0).agg(sum(col(cl1).cast("double")).alias(cl1),
                                  sum(col(cl2).cast("double")).alias(cl2))
    
    path = "/Users/diakite/Downloads/nfs_sunshine/random_refonte/" + path_detected.split("/")[-1].replace("_*", "")
    
    print(path)
    
    df_final.coalesce(1).write.format("csv").mode("overwrite").option("header", "true").option("delimiter", ";").csv(path)
    
    
    
    

In [42]:
rms_rs_files("/Users/diakite/Downloads/random_refonte 2/postes_telephoniques_10000_*.csv")

/Users/diakite/Downloads/random_refonte 2/postes_telephoniques_10000_*.csv
/Users/diakite/Downloads/random_refonte_csv_files/postes_telephoniques_10000.csv


In [36]:
path_detected = "/Users/diakite/Downloads/csv_files/carte_OBA_*.csv"

print(path_detected)

df = spark.read.option("header", "true").option("delimiter", ";").csv(path_detected)

df.show(5)

# , sum(col(cl2).cast("double")).alias(cl2)

/Users/diakite/Downloads/csv_files/carte_OBA_*.csv
+--------------+--------+
|    partenaire|quantity|
+--------------+--------+
|           SII|       2|
|          null|      99|
|  DISTRICENTRE|       2|
|APOCI SERVICES|       1|
|    Agence OCI|      60|
+--------------+--------+
only showing top 5 rows



In [38]:
cl0,cl1 = df.columns[0],df.columns[1]

df_final = df.groupBy(cl0).agg(sum(col(cl1).cast("double")).alias(cl1))

df_final.show()

+--------------+--------+
|    partenaire|quantity|
+--------------+--------+
|           SII|    26.0|
|        AMANDA|     2.0|
|          null|  1622.0|
|  DISTRICENTRE|    12.0|
|APOCI SERVICES|     6.0|
|           TID|     1.0|
|           IDT|     5.0|
|     LEAN DIST|     4.0|
|    Agence OCI|  1087.0|
|     REGHI TEL|     1.0|
|        MOSAIK|    19.0|
|           KTC|    10.0|
|     SUITE COM|    18.0|
|   LA COQUILLE|    14.0|
|          TKGS|    10.0|
|        AMANDA|     3.0|
|           AJS|    24.0|
+--------------+--------+



In [40]:
path = "/Users/diakite/Downloads/random_refonte_csv_files/" + path_detected.split("/")[-1].replace("_*", "")

print(path)

df_final.coalesce(1).write.format("csv").mode("overwrite").option("header", "true").option("delimiter", ";").csv(path)


/Users/diakite/Downloads/random_refonte_csv_files/carte_OBA.csv


In [13]:
import os

# specify the directory path
dir_path = "/Users/diakite/Downloads/nfs_sunshine-1/random_refonte"

# get all files in the directory
files = os.listdir(dir_path)

In [14]:
files

['equipements_internet_uniquement_20230321_sp.csv',
 'access_10_mb_20230321_sp.csv',
 'encaissement_facture_fixe_2_20230318_sp.csv',
 'migration_fibre_20230311_sp.csv',
 'terminal_famoco_20230321_sp.csv',
 'postes_telephoniques_10000_45000_20230304_sp.csv',
 'orange_wifi_nfg_max_30000_20230313_sp.csv',
 'encaissement_facture_fixe_1_20230305_sp.csv',
 'ter_50000_20230312_sp.csv',
 'encaissement_facture_fixe_2_20230306_sp.csv',
 'orange_wifi_intense_10000_20230321_sp.csv',
 'encaissement_facture_fixe_1_20230309_sp.csv',
 'must_300_mb_20230312_sp.csv',
 'postes_telephoniques_10000_45000_20230308_sp.csv',
 'migration_fibre_20230303_sp.csv',
 'orange_wifi_nfg_max_30000_20230301_sp.csv',
 'postes_telephoniques_10000_45000_20230316_sp.csv',
 'decouverte_50_mb_20230312_sp.csv',
 'encaissement_facture_fixe_2_20230314_sp.csv',
 'encaissement_facture_fixe_1_20230317_sp.csv',
 'access_10_mb_20230305_sp.csv',
 'equipements_internet_uniquement_20230305_sp.csv',
 'premium_200_mb_20230313_sp.csv',
 'o

In [15]:
import re


t = []

for file in files:
    if "csv" in file and "crc" not in file : 
        file = re.sub(r'\d{8}', '*', file)
        t.append(file)
        

In [16]:
t

['equipements_internet_uniquement_*_sp.csv',
 'access_10_mb_*_sp.csv',
 'encaissement_facture_fixe_2_*_sp.csv',
 'migration_fibre_*_sp.csv',
 'terminal_famoco_*_sp.csv',
 'postes_telephoniques_10000_45000_*_sp.csv',
 'orange_wifi_nfg_max_30000_*_sp.csv',
 'encaissement_facture_fixe_1_*_sp.csv',
 'ter_50000_*_sp.csv',
 'encaissement_facture_fixe_2_*_sp.csv',
 'orange_wifi_intense_10000_*_sp.csv',
 'encaissement_facture_fixe_1_*_sp.csv',
 'must_300_mb_*_sp.csv',
 'postes_telephoniques_10000_45000_*_sp.csv',
 'migration_fibre_*_sp.csv',
 'orange_wifi_nfg_max_30000_*_sp.csv',
 'postes_telephoniques_10000_45000_*_sp.csv',
 'decouverte_50_mb_*_sp.csv',
 'encaissement_facture_fixe_2_*_sp.csv',
 'encaissement_facture_fixe_1_*_sp.csv',
 'access_10_mb_*_sp.csv',
 'equipements_internet_uniquement_*_sp.csv',
 'premium_200_mb_*_sp.csv',
 'orange_wifi_intense_10000_*_sp.csv',
 'oweb_*_sp.csv',
 'encaissemt_factures_mobiles_1_*_sp.csv',
 'encaissemt_factures_mobiles_2_*_sp.csv',
 'terminal_famoco_*_s

In [17]:
d = list(set(t))

In [18]:
len(t) , len(d)

(693, 33)

In [10]:
for i in d :
    try :
        rms_rs_files(f"/Users/diakite/Downloads/nfs_sunshine/random_refonte/{i}")
    except :
        continue

In [42]:
import os
import shutil
import re

# set the root folder path and the destination folder path
root_folder = '/Users/diakite/Downloads/Users/diakite/Downloads/cvs_files/'
destination_folder = '/Users/diakite/Downloads/rf2/'

# iterate over all the subfolders in the root folder
for folder_name in os.listdir(root_folder):
    folder_path = os.path.join(root_folder, folder_name)
    
    # check if the item is a folder
    if os.path.isdir(folder_path):
        # get the dynamic part of the file name (if any)
        match = re.search(r'part-(.*?).csv', os.listdir(folder_path)[0])
        suffix = match.group(1) if match else ''
        
        # construct the new filename
        new_filename = folder_name + '_' + suffix + '.csv'
        
        # get the path to the input file
        input_file_path = os.path.join(folder_path, '_' + suffix + '.csv')
        
        # get the path to the output file
        output_file_path = os.path.join(destination_folder, new_filename)
        
        # rename the file and move it to the destination folder
        shutil.copy(input_file_path, output_file_path)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/diakite/Downloads/Users/diakite/Downloads/random_refonte 2/ter_50000_*.csv/_.csv'

In [31]:
import pandas as pd
import glob

def get_agg_month(source,patern) :

        # Define the pattern to match the filenames
        file_pattern = f'{source}/{patern}'

        # Get a list of matching file paths
        file_paths = glob.glob(file_pattern)

        # Create an empty list to store the dataframes
        dfs = []

        # Iterate over the file paths and read each file
        for file_path in file_paths:
            #continue
            print(file_path)
            try : 
                df = pd.read_csv(file_path, sep = ";")
                dfs.append(df)
            except :
                continue

        # Concatenate all the dataframes into a single dataframe
        combined_df = pd.concat(dfs, ignore_index=True)
        print(combined_df.columns)

        cl0,cl1,cl2 = list(combined_df.columns)[0],list(combined_df.columns)[1],list(combined_df.columns)[2]

        combined_df[f"{cl1}"] = combined_df[f"{cl1}"].astype(float)
        combined_df[f"{cl2}"] = combined_df[f"{cl2}"].astype(float)

        df_final = combined_df.groupby(f'{cl0}').agg({
            f'{cl1}': 'sum',
            f'{cl2}': 'sum'
        }).rename(columns={f'{cl1}': f'{cl1}', f'{cl2}': f'{cl2}'}).reset_index()


        path = "/Users/diakite/Downloads/" + file_pattern.split("/")[-1].replace("_*", "")

        # Now you have a single dataframe with the data from all the files
        df_final.head()
        
        df_final.to_csv(path, index = False)

In [32]:
for i in d :
    try :
        get_agg_month("/Users/diakite/Downloads/nfs_sunshine-2/random_refonte", i)
    except :
        continue

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/recla_justifi_20230326_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/recla_justifi_20230324_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/recla_justifi_20230328_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/recla_justifi_20230330_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/recla_justifi_20230327_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/recla_justifi_20230325_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/recla_justifi_20230329_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/postes_telephoniques_10000_20230323_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/postes_telephoniques_10000_45000_20230330_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/postes_telephoniques_10000_20230327_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/postes_telephoniques_10000_45000_20230326_sp.csv
/

Index(['partenaire', 'sum_quantity', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/domino_4G_20230327_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/domino_4G_20230329_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/domino_4G_20230325_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/domino_4G_20230323_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/domino_4G_20230326_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/domino_4G_20230328_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/domino_4G_20230324_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/domino_4G_20230330_sp.csv
Index(['partenaire', 'sum_quantity', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/terminal_famoco_20230323_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/terminal_famoco_20230327_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/ra

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/offre_mykbine_20230326_sp.csv
Index(['distributeur', 'sum_ca_kbine_om', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/decouverte_50_mb_20230324_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/decouverte_50_mb_20230328_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/decouverte_50_mb_20230326_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/decouverte_50_mb_20230330_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/decouverte_50_mb_20230325_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/decouverte_50_mb_20230329_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/decouverte_50_mb_20230327_sp.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/decouverte_50_mb_20230323_sp.csv
Index(['partenaire', 'count_offre', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/orange_wifi_inte

In [34]:
for i in d :
    try :
        get_agg_month("/Users/diakite/Downloads/nfs_sunshine-2/random_refonte", "2_c2s_*.csv")
    except :
        continue

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230307.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230313.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230312.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230306.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230310.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230304.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230305.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230311.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230329.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230315.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230301.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230314.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230328.csv
/Users/diakite/Downloads/nfs_sunshine-

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230324.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230330.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230308.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230320.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230321.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230309.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230323.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230322.csv
Index(['nom_partenaire', 'select_calculer_a', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230307.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230313.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230312.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230306.csv
/Users/diakite/Downloads/nfs_sunshi

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230315.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230301.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230314.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230328.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230302.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230316.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230317.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230303.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230326.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230327.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230325.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230319.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230318.csv
/Users/diakite/Downloads/nfs_sunshine-

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230322.csv
Index(['nom_partenaire', 'select_calculer_a', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230307.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230313.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230312.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230306.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230310.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230304.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230305.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230311.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230329.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230315.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230301.csv
/Users/diakite/Downloads/nfs_sunshi

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230327.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230325.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230319.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230318.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230324.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230330.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230308.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230320.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230321.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230309.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230323.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230322.csv
Index(['nom_partenaire', 'select_calculer_a', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshi

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230301.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230314.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230328.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230302.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230316.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230317.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230303.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230326.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230327.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230325.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230319.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230318.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230324.csv
/Users/diakite/Downloads/nfs_sunshine-

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230322.csv
Index(['nom_partenaire', 'select_calculer_a', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230307.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230313.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230312.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230306.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230310.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230304.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230305.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230311.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230329.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230315.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230301.csv
/Users/diakite/Downloads/nfs_sunshi

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230319.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230318.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230324.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230330.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230308.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230320.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230321.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230309.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230323.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230322.csv
Index(['nom_partenaire', 'select_calculer_a', 'commission'], dtype='object')
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230307.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230313.csv
/Users/diakite/Downloads/nfs_sunshi

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230301.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230314.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230328.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230302.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230316.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230317.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230303.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230326.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230327.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230325.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230319.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230318.csv
/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230324.csv
/Users/diakite/Downloads/nfs_sunshine-

/Users/diakite/Downloads/nfs_sunshine-2/random_refonte/2_c2s_20230322.csv
Index(['nom_partenaire', 'select_calculer_a', 'commission'], dtype='object')
